# TEIS Referral Project

# Table of Contents

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from scipy.stats import ttest_ind_from_stats
from statsmodels.stats.proportion import proportions_ztest

c:\Users\jrior\anaconda3\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.0)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


## Dictionaries

In [2]:
poe_dict = {
    
    'FT': 'First Tennessee',
    'UC': 'Upper Cumberland',
    'NW':'Northwest',
    'ET': 'East Tennessee',
    'GN': 'Greater Nashville',
    'SW': 'Southwest',
    'SE': 'Southeast',
    'SC': 'South Central',
    'MD': 'Memphis Delta'

}

## Methods

In [ ]:
def calc_rerefs( self , group, proportion = False, as_percentage = False):
    """
    Bound method on pandas Dataframe.
    Takes input dataframe instance and returns count or proportion of re-referrals according to input group.
    """
    
    # get rows of children who get re-referred
    rerefs = self[self.duplicated(subset = 'Child ID', keep = False)]

    # calculate count or proportion by input group
    if proportion:
        group_calc = rerefs.groupby(group)['Child ID'].count() / self.groupby(group)['Child ID'].count()
    else:
        group_calc = rerefs.groupby(group)['Child ID'].count()

    # return group calculation with optional formatting
    if as_percentage:
        return group_calc.map(lambda x: '{:,.2%}'.format(x))
    else:
        return group_calc

pd.DataFrame.calc_rerefs = calc_rerefs

## Functions

In [3]:
def ztest_against_pop(series):
    return proportions_ztest(count = [series['sum'], overall_referral_successes], 
                             nobs = [series['count'], overall_referral_counts])

def ttest_against_pop(series):
    return ttest_ind_from_stats(mean1 = series['mean'], 
                        std1 = series['std'], 
                        nobs1 = series['count'],
                        mean2 = overall_mean_age_at_referral, 
                        std2 = overall_std_age_at_referral, 
                        nobs2 = overall_count_age_at_referral
                    )


def test_to_df(df, func):
    test_stats = [func(row)[0] for poe, row in df.iterrows()]
    pvals = [func(row)[1] for poe, row in df.iterrows()]
    highlows = ['high' if stat > 0  else 'low' for stat in test_stats]
    significances = [pval < 0.05 for pval in pvals]

    new_df = (df.assign(test_stat = test_stats,
                                pval = pvals,
                                high_or_low = highlows,
                                significant = significances)
                        .sort_values('test_stat', ascending = False)
            )

    return new_df

## Get data

In [4]:
summary = pd.read_excel('../data/TEIS Referral Data for NSS pulled 7-1-22.xlsx')
summary.head()

,POE,Child ID,DOB,County Name,County SES,Child Phase,Active,Service Coordinator,Notification Date,Parent Consent Date,Referral Source Type Name,Initial Eligibility,Initial Eligibility Date,Initial IFSP Date,Latest IFSP Date,Exit Reason,Exit Date
0,ET,411812,2016-12-14,Blount,Transitional,Notification,I,Charity Decker,2016-12-16,NaT,DCS,NaN,NaT,NaT,NaT,Unable to contact,2016-12-29
1,ET,417398,2015-04-06,Knox,Transitional,Eligibility,I,Jackie Campbell,2017-04-24,2017-05-02,Parent,Ineligible,2017-05-16,NaT,NaT,Ineligible for Part C,2017-05-16
2,ET,419400,2016-03-04,Roane,Transitional,Notification,I,Kelly Brackney,2017-06-06,NaT,DCS,NaN,NaT,NaT,NaT,Parent decline,2017-06-30
3,ET,397571,2015-05-11,Knox,Transitional,Eligibility,I,Wendy Burton,2017-05-24,2017-06-13,Other,Ineligible,2017-06-16,NaT,NaT,Ineligible for Part C,2017-06-16
4,ET,404648,2015-09-23,Knox,Transitional,IFSP,I,Sarah Scott,2016-07-18,2016-07-28,Other,Eligible,2016-08-01,2016-08-16,2018-07-24,618 - Part B eligibility not determined,2018-09-21


In [5]:
monthly = pd.read_excel('../data/TEIS Referral Data for NSS Compiled Monthly.xlsx')
monthly.head()

,POE,Child ID,DOB,County Name,County SES,Child Phase,Active,Service Coordinator,Notification/ Referral Date,Parent Consent Date,...,Service Coordinator Counter,Fiscal Year.1,Child Count,2012 Child Count,2013 Child Count,third DOB,Late Referral,Qtr,QTR,Referral Source Category
0,ET,403339,2014-08-09,Blount,Transitional,Eligibility,A,Kristi Borer,2016-07-01,2016-07-01,...,1,2016,1,NaN,NaN,2017-08-09,NaN,1,Qtr 1,NaN
1,FT,404085,2013-07-31,Unicoi,At-Risk,Notification,I,Jennifer Terranera - 45 days,2016-07-01,NaT,...,1,2016,1,NaN,NaN,2016-07-31,1.0,1,Qtr 1,NaN
2,FT,403623,2016-02-05,Sullivan,Transitional,Eligibility,A,Amy Talbert,2016-07-01,2016-07-20,...,1,2016,1,NaN,NaN,2019-02-05,NaN,1,Qtr 1,NaN
3,FT,404157,2015-12-06,Sullivan,Transitional,IFSP,A,Candice Cradic,2016-07-01,2016-07-22,...,1,2016,1,NaN,NaN,2018-12-06,NaN,1,Qtr 1,NaN
4,FT,404154,2015-04-18,Hawkins,At-Risk,IFSP,A,Kathy Jeffries,2016-07-01,2016-07-18,...,1,2016,1,NaN,NaN,2018-04-18,NaN,1,Qtr 1,NaN


How many Child IDs are in the monthly dataframe and not the summary df? Vice-versa?

In [6]:
# monthly Child IDs not in summary
month_not_sum = (~monthly['Child ID'].isin(summary['Child ID'])).sum()
month_not_sum

175

In [7]:
# summary Child IDs not in monthly
sum_not_month = (~summary['Child ID'].isin(monthly['Child ID'])).sum()
sum_not_month

1793

## Overview

In [8]:
summary.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96580 entries, 0 to 96579
Data columns (total 17 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   POE                        96580 non-null  object        
 1   Child ID                   96580 non-null  int64         
 2   DOB                        96580 non-null  datetime64[ns]
 3   County Name                96576 non-null  object        
 4   County SES                 96576 non-null  object        
 5   Child Phase                96580 non-null  object        
 6   Active                     96580 non-null  object        
 7   Service Coordinator        96579 non-null  object        
 8   Notification Date          96580 non-null  datetime64[ns]
 9   Parent Consent Date        70026 non-null  datetime64[ns]
 10  Referral Source Type Name  96580 non-null  object        
 11  Initial Eligibility        68221 non-null  object        
 12  Init

In [9]:
monthly.columns

Index(['POE', 'Child ID', 'DOB', 'County Name', 'County SES', 'Child Phase',
       'Active', 'Service Coordinator', 'Notification/ Referral Date',
       'Parent Consent Date', 'Referral Source Type Name',
       'Initial Eligibility', 'Initial Eligibility Date', 'Initial IFSP Date',
       'Latest IFSP Date', 'Exit Reason', 'Exit Date', 'Fiscal Year',
       'Notification Month', 'Tenn Region', 'Referral Count',
       'Evaluation Count', 'Eligibility Count', 'IFSP Count',
       'Service Coordinator Counter', 'Fiscal Year.1', 'Child Count',
       '2012 Child Count', '2013 Child Count', 'third DOB', 'Late Referral',
       'Qtr', ' QTR ', 'Referral Source Category'],
      dtype='object')

Based simply on the number of unique Child IDs in each table, there's some discrepancy in tracking.

In [10]:
print('summary distinct Child IDs:', summary['Child ID'].nunique(), 
      '\nmonthly distinct Child IDs:', monthly['Child ID'].nunique(),
      '\ndifference:', summary['Child ID'].nunique() - monthly['Child ID'].nunique())

summary distinct Child IDs: 96580 
monthly distinct Child IDs: 94950 
difference: 1630


## Q1
- What is the rate of re-referral?
- How many children have multiple referrals 
    - by year
    - by Point of Entry Office (POE)

In [11]:
def reref_calc(df, proportion = False, as_percentage = False):
    """Use value counts over 1 to determine how many or what proportion of children get re-referrals."""
    if proportion:
        calc = (df['Child ID'].value_counts() > 1).mean()
    else:
        calc = (df['Child ID'].value_counts() > 1).sum()

    if as_percentage:
        return "{:.2%}".format(calc)
    else:
        return calc


### What is the rate of re-referral?

In [12]:
print(reref_calc(monthly, proportion = True, as_percentage=True), "of all children get re-referrals.")

16.08% of all children get re-referrals.


### How many children have multiple referrals by year and by POE?

This approach first groups then looks for re-referrals. It therefore looks for re-referrals within each group rather than across the whole data set.

In [13]:
# by fiscal year
monthly.groupby('Fiscal Year').apply(reref_calc)
monthly.groupby('Fiscal Year').apply(reref_calc, proportion = True, as_percentage = True)

# by poe
monthly.groupby('POE').apply(reref_calc).sort_values(ascending = False)
monthly.groupby('POE').apply(reref_calc, proportion = True, as_percentage = True).sort_values(ascending=False)

# by both poe and fiscal year
monthly.groupby(['POE','Fiscal Year']).apply(reref_calc).sort_index()
monthly.groupby(['POE', 'Fiscal Year']).apply(reref_calc, proportion = True, as_percentage = True).sort_index()

POE  Fiscal Year
ET   2016-2017       8.61%
     2017-2018       8.46%
     2018-2019       7.59%
     2019-2020       8.06%
     2020-2021       8.75%
     2021-2022       7.70%
FT   2016-2017       8.02%
     2017-2018       8.96%
     2018-2019       8.44%
     2019-2020       7.06%
     2020-2021       6.90%
     2021-2022       7.72%
GN   2016-2017      10.35%
     2017-2018       9.10%
     2018-2019       6.20%
     2019-2020       8.49%
     2020-2021      10.59%
     2021-2022       9.62%
MD   2016-2017      11.81%
     2017-2018      11.14%
     2018-2019      10.89%
     2019-2020       8.20%
     2020-2021       8.26%
     2021-2022       8.15%
NW   2016-2017       7.29%
     2017-2018       8.13%
     2018-2019       6.28%
     2019-2020       8.40%
     2020-2021       8.03%
     2021-2022       7.75%
SC   2016-2017       8.77%
     2017-2018       7.87%
     2018-2019       5.59%
     2019-2020       9.51%
     2020-2021       9.02%
     2021-2022       7.94%
SE   2016-2

If we instead want to keep duplicates across the data set and then count, we go this route (and get larger numbers).

(Note that this includes any referral of a child who was re-referred at any point. So if a child has two referrals, both are in this set.)

Get re-referrals by fiscal year.

In [15]:
monthly.calc_rerefs(group = 'Fiscal Year')

Fiscal Year
2016-2017    4297
2017-2018    5594
2018-2019    6014
2019-2020    6337
2020-2021    6505
2021-2022    4826
Name: Child ID, dtype: int64

In [16]:
monthly.calc_rerefs(group = 'Fiscal Year', proportion=True, as_percentage=True)

Fiscal Year
2016-2017    26.05%
2017-2018    31.40%
2018-2019    30.07%
2019-2020    33.38%
2020-2021    31.35%
2021-2022    25.12%
Name: Child ID, dtype: object

Get re-referrals by POE

In [17]:
monthly.calc_rerefs(group = 'POE')

POE
ET    6161
FT    3358
GN    6215
MD    5695
NW    1986
SC    3966
SE    2018
SW    1746
UC    2428
Name: Child ID, dtype: int64

In [18]:
monthly.calc_rerefs(group = 'POE', proportion=True, as_percentage=True)

POE
ET    30.46%
FT    30.14%
GN    31.14%
MD    32.99%
NW    30.53%
SC    27.29%
SE    23.53%
SW    28.94%
UC    26.91%
Name: Child ID, dtype: object

Get re-referrals by POE and fiscal year

In [19]:
monthly.calc_rerefs(group = ['POE', 'Fiscal Year'])

POE  Fiscal Year
ET   2016-2017       857
     2017-2018      1069
     2018-2019      1148
     2019-2020      1159
     2020-2021      1140
     2021-2022       788
FT   2016-2017       491
     2017-2018       693
     2018-2019       681
     2019-2020       583
     2020-2021       510
     2021-2022       400
GN   2016-2017       796
     2017-2018       914
     2018-2019       927
     2019-2020      1160
     2020-2021      1408
     2021-2022      1010
MD   2016-2017       800
     2017-2018      1040
     2018-2019      1175
     2019-2020       956
     2020-2021       964
     2021-2022       760
NW   2016-2017       230
     2017-2018       338
     2018-2019       352
     2019-2020       415
     2020-2021       373
     2021-2022       278
SC   2016-2017       483
     2017-2018       624
     2018-2019       618
     2019-2020       816
     2020-2021       834
     2021-2022       591
SE   2016-2017       170
     2017-2018       273
     2018-2019       398
     201

In [20]:
monthly.calc_rerefs(group = ['POE', 'Fiscal Year'], proportion=True, as_percentage=True)

POE  Fiscal Year
ET   2016-2017      27.17%
     2017-2018      32.15%
     2018-2019      32.21%
     2019-2020      34.26%
     2020-2021      32.38%
     2021-2022      24.03%
FT   2016-2017      26.71%
     2017-2018      34.09%
     2018-2019      33.66%
     2019-2020      31.62%
     2020-2021      28.76%
     2021-2022      24.51%
GN   2016-2017      28.64%
     2017-2018      31.60%
     2018-2019      26.92%
     2019-2020      33.71%
     2020-2021      36.53%
     2021-2022      28.47%
MD   2016-2017      31.57%
     2017-2018      38.79%
     2018-2019      36.97%
     2019-2020      35.12%
     2020-2021      30.27%
     2021-2022      25.67%
NW   2016-2017      23.93%
     2017-2018      31.68%
     2018-2019      31.91%
     2019-2020      36.40%
     2020-2021      31.48%
     2021-2022      26.48%
SC   2016-2017      24.64%
     2017-2018      28.89%
     2018-2019      24.50%
     2019-2020      32.14%
     2020-2021      29.83%
     2021-2022      23.13%
SE   2016-2

## Q2
A referral is considered a "successful" referral if it results in an Individualized Family Service Plan (IFSP). 
- Are there referral sources that have had higher rates of successful referrals? 

You can determine whether a referral was successful by looking at whether the Initial IFSP Date column is filled in or whether the Child Phase column is equal to IFSP.

In [21]:
for col in monthly.filter(regex='.*Date.*').columns:
    monthly[col] = pd.to_datetime(monthly[col], errors = 'coerce')

In [22]:
monthly['Successful Referral'] = (monthly['Initial IFSP Date'].notna()) | (monthly['Child Phase'] == 'IFSP')

In [23]:
overall_referral_success_rate = monthly['Successful Referral'].mean()
overall_referral_successes = monthly['Successful Referral'].sum()
overall_referral_counts = monthly['Successful Referral'].count()
print("The overall referral success rate is", '{:.2%}'.format(overall_referral_success_rate))

The overall referral success rate is 7.07%


In [24]:
source_referral_success_rate = (monthly.groupby('Referral Source Type Name')
                                        ['Successful Referral']
                                        .agg(['sum', 'count', 'mean', 'std'])
                                        .sort_values('mean', ascending = False)
)

In [25]:
source_referral_significances = test_to_df(source_referral_success_rate, ztest_against_pop)
source_referral_significances[source_referral_significances['significant']]

,sum,count,mean,std,test_stat,pval,high_or_low,significant
Referral Source Type Name,,,,,,,,
Parent,2494,21530,0.115838,0.320038,22.636088,1.912868e-113,high,True
Foster Parent,72,413,0.174334,0.379857,8.178266,2.879585e-16,high,True
PT,85,609,0.139573,0.346828,6.593837,4.286034e-11,high,True
DMRS,1,1,1.000000,NaN,3.624611,2.893972e-04,high,True
Family and Friends,55,507,0.108481,0.311294,3.305154,9.492438e-04,high,True
Other,221,2561,0.086294,0.280853,3.032475,2.425571e-03,high,True
Speech Therapist,64,638,0.100313,0.300653,2.903954,3.684823e-03,high,True
Dept. of Health,212,3631,0.058386,0.234504,-2.861933,4.210655e-03,low,True
NICU,184,3232,0.056931,0.231746,-3.023765,2.496500e-03,low,True


## Q3 
- How do POEs compare on these metrics:
    - a low proportion of referrals
    - a high average age of referrals
    - a low percent of total referrals resulting in an IFSP
    - a lower percent of eligible referrals resulting in an IFSP
- Which POEs are of concern on the basis of one or more of the listed criteria?

Low proportion of referrals

In [27]:
monthly['POE'].value_counts(normalize=True)

ET    0.178587
GN    0.176203
MD    0.152407
SC    0.128311
FT    0.098388
UC    0.079669
SE    0.075713
NW    0.057445
SW    0.053278
Name: POE, dtype: float64

Average age of referrals

In [28]:
monthly['Age Days'] = (monthly['Notification/ Referral Date'] - monthly['DOB']).dt.days
overall_mean_age_at_referral = monthly['Age Days'].mean()
overall_std_age_at_referral = monthly['Age Days'].std()
overall_count_age_at_referral = monthly['Age Days'].count()

overall_mean_age_at_referral

467.2186374231829

In [29]:
age_by_poe = (monthly.groupby('POE')
                    ['Age Days']
                    .agg(['count', 'mean', 'std'])
                    .sort_values('mean', ascending = False)
        )

In [31]:
age_by_poe_significances = test_to_df(age_by_poe, ttest_against_pop)
age_by_poe_significances

,count,mean,std,test_stat,pval,high_or_low,significant
POE,,,,,,,
SC,14532,518.637421,305.777595,18.302696,9.817826e-75,high,True
GN,19956,501.435157,312.301587,13.960336,2.924466e-44,high,True
UC,9023,473.284274,320.632159,1.730263,8.358590e-02,high,False
MD,17261,467.598169,319.341684,0.145011,8.847023e-01,high,False
ET,20226,451.421339,317.583887,-6.466493,1.006478e-10,low,True
SW,6034,437.171197,327.995627,-7.088257,1.365605e-12,low,True
SE,8575,435.652362,331.303092,-8.773355,1.755953e-18,low,True
NW,6506,428.345681,329.588518,-9.499875,2.138239e-21,low,True
FT,11143,425.316701,326.959108,-13.145730,1.913196e-39,low,True


## Q4
Look for patterns in referrals and re-referrals on the basis of the following:
- Age
- Referral source
- Socioeconomic status (County SES)


## Q5
TEIS provides services to eligible infants and toddlers from birth to age 3. 
- Are there any POE with high rates of early closure (a child exiting before reaching age 3)? 
- Are there any service coordinators with high rates of early closure (a child exiting before reaching age 3)?

For this question, only consider cases which have reached the IFSP phase.